In [1]:
# -*- coding: utf-8 -*-

# 1.加载包

In [2]:
import nltk
import pandas as pd
import numpy as np
import re
import codecs

# 2.加载点赞、评论数均大于5的最受欢迎问题

In [123]:
popularquestion=pd.read_excel('/Users/taolong/数据创新比赛/01最受欢迎问题.xlsx')
#popularquestion.columns=['number','questiontext','thumbsup','review']
popularquestion.head()

,number,questiontext,thumbup,review,topic,province,city,phase,问题标题
0,1,变形金刚用的能量块，是否能够实现？,7,9,能源科学,上海市,上海市,小学,能源
1,2,人们每天都产生许多生活垃圾，带来许多环境问题，可不可以利用起来（垃圾中有许多资源），利用特殊...,6,11,能源科学,江西省,赣州市,高中,为什么不发明利用生活垃圾作燃料的汽车？
2,3,既然太阳能、风力是取之不尽用之不竭的，为什么现在还有火力发电？这不是浪费资源、污染环境吗？,6,6,能源科学,浙江省,宁波市,初中,为什么不全部采用清洁能源？
3,4,世界上缺什么东西最可怕,5,5,能源科学,NaN,NaN,NaN,物理
4,5,如果有一天太阳没有了 人类还能生存下去吗?,9,9,能源科学,上海市,上海市,小学,天文学


In [124]:
cleantext=popularquestion.drop(['topic','province','city','phase','问题标题'] ,axis=1)

In [125]:
cleantext.head()

,number,questiontext,thumbup,review
0,1,变形金刚用的能量块，是否能够实现？,7,9
1,2,人们每天都产生许多生活垃圾，带来许多环境问题，可不可以利用起来（垃圾中有许多资源），利用特殊...,6,11
2,3,既然太阳能、风力是取之不尽用之不竭的，为什么现在还有火力发电？这不是浪费资源、污染环境吗？,6,6
3,4,世界上缺什么东西最可怕,5,5
4,5,如果有一天太阳没有了 人类还能生存下去吗?,9,9


# 3.正则表达式替换

In [126]:
def standardtext(df,text):
    df[text]=df[text].str.replace('/^[+-]?(0|([1-9]\d*))(\.\d+)?$/','',10)#数字
    df[text]=df[text].str.replace('(?i)^(?![\d\-]+$)[a-z\d\-+*/_]+$','',10)#匹配加减乘除
   # df[text]=df[text].str.replace('','',10)
    return df
cleantext1=standardtext(cleantext,'questiontext')
cleantext1.to_csv('cleantext1.csv')

In [127]:
cleantext1.head()

,number,questiontext,thumbup,review
0,1,变形金刚用的能量块，是否能够实现？,7,9
1,2,人们每天都产生许多生活垃圾，带来许多环境问题，可不可以利用起来（垃圾中有许多资源），利用特殊...,6,11
2,3,既然太阳能、风力是取之不尽用之不竭的，为什么现在还有火力发电？这不是浪费资源、污染环境吗？,6,6
3,4,世界上缺什么东西最可怕,5,5
4,5,如果有一天太阳没有了 人类还能生存下去吗?,9,9


# 4.结巴分词

In [159]:
def cutword(x):
    seg=jieba.cut(x,cut_all=False)
    return ' '.join(seg)
cleantext1['questiontext']=cleantext1['questiontext'].apply(cutword)

In [160]:
cleantext1.head()

,number,questiontext,thumbup,review
0,1,变形金刚 用 的 能量 块 ， 是否 能够 实现 ？,7,9
1,2,人们 每天 都 产生 许多 生活 垃圾 ， 带来 许多 环境 问题 ， 可不可以 利用 起来...,6,11
2,3,既然 太阳能 、 风力 是 取之不尽 用之不竭 的 ， 为什么 现在 还有 火力发电 ？ 这...,6,6
3,4,世界 上 缺什么 东西 最 可怕,5,5
4,5,如果 有 一天 太阳 没有 了 人类 还 能 生存 下去 吗 ?,9,9


# 5.去停用词

In [161]:
stopword=pd.read_csv('/Users/taolong/stopwords.txt',names=['w'],encoding='utf-8',sep='aaa')
stopword.head()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,w
0,的
1,啊
2,阿
3,哎
4,哎呀


In [162]:
def remove_stopword(sen):
    segs=[word for word in sen.split(' ') if word not in list(stopword.w) and len(word)>1]
    return ' '.join(segs)

In [163]:
cleantext1['newtext']=cleantext1['questiontext'].apply(remove_stopword)

In [164]:
cleantext1.head()

,number,questiontext,thumbup,review,newtext
0,1,变形金刚 用 的 能量 块 ， 是否 能够 实现 ？,7,9,变形金刚 能量
1,2,人们 每天 都 产生 许多 生活 垃圾 ， 带来 许多 环境 问题 ， 可不可以 利用 起来...,6,11,生活 垃圾 环境 可不可以 利用 垃圾 中有 资源 利用 微生物 发酵 甲烷 气体 汽车 燃...
2,3,既然 太阳能 、 风力 是 取之不尽 用之不竭 的 ， 为什么 现在 还有 火力发电 ？ 这...,6,6,太阳能 风力 取之不尽 用之不竭 火力发电 浪费资源 污染环境
3,4,世界 上 缺什么 东西 最 可怕,5,5,世界 缺什么 东西 可怕
4,5,如果 有 一天 太阳 没有 了 人类 还 能 生存 下去 吗 ?,9,9,太阳 人类 生存


# 6.词频矩阵

In [169]:
from sklearn.feature_extraction.text import  CountVectorizer
#将文本中的词语转换为词频矩阵
vectorizer = CountVectorizer(analyzer='word')
#计算个词语出现的次数
X=vectorizer.fit_transform(cleantext1['newtext'].values.astype('U'))
X.shape


(449, 1274)

In [213]:
words=vectorizer.get_feature_names()

In [214]:
type(words)

list

# 7.tf-idf加权

In [166]:
from sklearn.feature_extraction.text import  TfidfVectorizer
transformer=TfidfVectorizer(max_features=100)
transformer.fit_transform(cleantext1['newtext'].values.astype('U'))

tfidf.shape

(449, 100)

In [167]:
tfidf_df=pd.SparseDataFrame(tfidf, columns=transformer.get_feature_names(), index=range(1,450), default_fill_value=0)


In [120]:
tfidf_df.head(20)

,10,100,11,20,30,36,sys,一共,一半,一只,...,装置,解决,请问,起飞,酢浆草,闪电,雷声,颜色,鸡蛋,黑洞
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.266509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 8.清洗过后文本内容放进语料库

In [220]:
corpus_text=cleantext1['newtext']
corpus_text.head()
corpus_text.to_csv('corpus_text.csv')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


In [221]:
corpus = []
for line in open('corpus_text.csv','r').readlines():
    corpus.append(line)

In [246]:
corpus

['0,变形金刚 能量\n',
 '1,生活 垃圾 环境 可不可以 利用 垃圾 中有 资源 利用 微生物 发酵 甲烷 气体 汽车 燃料 生活 垃圾 宝贵 资源 解决 生活 垃圾 汽车 越远 解决 生活 垃圾 资源 利用\n',
 '2,太阳能 风力 取之不尽 用之不竭 火力发电 浪费资源 污染环境\n',
 '3,世界 缺什么 东西 可怕\n',
 '4,太阳 人类 生存\n',
 '5,太阳 能源 发展 瓶颈\n',
 '6,柠檬 发电\n',
 '7,王国维 人间 词画 提及 之境 泪眼 花花 不语 之境 悠然 南山 视角 分辨\n',
 '8,全民 抖音 当今 社会 奇观 抖音 用户 低龄化 倾向 越来越 中小学生 乐衷于 抖音 中学生 吸引 众多 中小学生 中小学生 正确认识 理性 抖音 这是 关注\n',
 '9,宇宙 温度 极高 压强 奇点 爆炸 奇点 奇点 物质 世界 矛盾\n',
 '10,东北 经济 落后 南方\n',
 '11,强占 东西 非常高兴 违法\n',
 '12,面对 信息 过剩 自问自答 面对 屏幕 无能为力 一行 本来 介绍 信息 过剩 解释 本书 内容 本书 近代 依然 试用 文选 尼尔 兹曼 娱乐 至死 AMUSING OURSELVES TO DEATH 第五章 躲猫猫 世界 问问 一系列 明白 上文 中东 冲突 解决 通货膨胀\n',
 '13,滴滴 顺风 发生\n',
 '14,辩证 看待 盗版 是因为 买不起 正版 理直气壮 现象\n',
 '15,变强\n',
 '16,17 美国 实施 中国 2000 亿美元 商品 征加 关税 措施 18 中国 反制 美国 600 亿美元 商品 征加 关税 贸易战 真的\n',
 '17,掉头发\n',
 '18,真的 有龙\n',
 '19,大脑 想象 一副 儿时 记忆犹新 画面 画面 这幅 画面 正面 背影 诡异 明明 看不见 场中 视角 视角 sys 1004\n',
 '20,牛顿 爱因斯坦 超级 科学家 晚年 热衷于 宗教 探寻 科学 真谛\n',
 '21,国家 素质教育 规范 校外 培训 同学 参加 校外 培训 提高 成绩 手段\n',
 '22,人口数量 不增 不减 平均 妈妈 生育 几个 孩子\n',
 '23,费孝通 生育 制度 一书中 详细 描述 中国 近代 农

In [247]:
#vectorizer1=CountVectorizer()
#keywords=vectorizer1.fit_transform(corpus)
#print(keywords)
#keywords

### 注：上文生成的corpus为list，将其转化为str

In [248]:
type(corpus)

list

In [249]:
text123=''.join(corpus)

In [250]:
type(text123)

str

In [251]:
text123

'0,变形金刚 能量\n1,生活 垃圾 环境 可不可以 利用 垃圾 中有 资源 利用 微生物 发酵 甲烷 气体 汽车 燃料 生活 垃圾 宝贵 资源 解决 生活 垃圾 汽车 越远 解决 生活 垃圾 资源 利用\n2,太阳能 风力 取之不尽 用之不竭 火力发电 浪费资源 污染环境\n3,世界 缺什么 东西 可怕\n4,太阳 人类 生存\n5,太阳 能源 发展 瓶颈\n6,柠檬 发电\n7,王国维 人间 词画 提及 之境 泪眼 花花 不语 之境 悠然 南山 视角 分辨\n8,全民 抖音 当今 社会 奇观 抖音 用户 低龄化 倾向 越来越 中小学生 乐衷于 抖音 中学生 吸引 众多 中小学生 中小学生 正确认识 理性 抖音 这是 关注\n9,宇宙 温度 极高 压强 奇点 爆炸 奇点 奇点 物质 世界 矛盾\n10,东北 经济 落后 南方\n11,强占 东西 非常高兴 违法\n12,面对 信息 过剩 自问自答 面对 屏幕 无能为力 一行 本来 介绍 信息 过剩 解释 本书 内容 本书 近代 依然 试用 文选 尼尔 兹曼 娱乐 至死 AMUSING OURSELVES TO DEATH 第五章 躲猫猫 世界 问问 一系列 明白 上文 中东 冲突 解决 通货膨胀\n13,滴滴 顺风 发生\n14,辩证 看待 盗版 是因为 买不起 正版 理直气壮 现象\n15,变强\n16,17 美国 实施 中国 2000 亿美元 商品 征加 关税 措施 18 中国 反制 美国 600 亿美元 商品 征加 关税 贸易战 真的\n17,掉头发\n18,真的 有龙\n19,大脑 想象 一副 儿时 记忆犹新 画面 画面 这幅 画面 正面 背影 诡异 明明 看不见 场中 视角 视角 sys 1004\n20,牛顿 爱因斯坦 超级 科学家 晚年 热衷于 宗教 探寻 科学 真谛\n21,国家 素质教育 规范 校外 培训 同学 参加 校外 培训 提高 成绩 手段\n22,人口数量 不增 不减 平均 妈妈 生育 几个 孩子\n23,费孝通 生育 制度 一书中 详细 描述 中国 近代 农村 社会制度 代表性 婚姻制度 生育 制度 双系 抚育 家庭 组织 父母 两方 亲属关系 农村 孩子 出生 接受 父母 舅舅 多方 社会关系 教育 渐渐 长大 更会 接受 社会性 断乳 合适 年龄 后会 媒人 物色 对象 在内 外婚

# 9.关键词抽取

## 9.1基于tf-idf算法

### 提取前50个关键词，词性限制在“地名”、“机构团体”、“其他专名”和“形容词”

In [245]:
#jieba分词器通过词频获取关键词
import jieba
import jieba.analyse
  
keywords = jieba.analyse.extract_tags(text123, topK=50, withWeight=True, allowPOS=('ns','nt','nz','a'))
#地名、机构团体、其他专名、形容词
#keywords=jieba.analyse.extract_tags(text123,topK=20)

# print(type(keywords))
# <class 'list'>
 
for item in keywords:
    print(item[0],item[1])

黑洞 0.7276724282273275
恐龙 0.4731252134932759
星星 0.35801578760215513
太阳 0.3221337804481035
硅酸盐 0.31202232372620686
泰坦尼克号 0.28946450304051724
可显 0.20611668108448275
东西 0.20574500865603448
有害 0.2035342859925
硅酸钠 0.19297633536034484
城市 0.17191890964206896
可乐 0.16907470844396552
冰山 0.16026737010327585
甲烷 0.1566981602862069
中国 0.15658555275827585
天文学 0.1377565691024138
越夏 0.11385802130517242
哥哥 0.10962869291
甲午海战 0.10595896327155173
成熟 0.1057631164475862
霸王龙 0.10438722571293103
真实世界 0.10305834054224138
虚拟世界 0.09916191706206896
直角三角形 0.09491643012155172
美人鱼 0.09280330548275861
阿拉伯数字 0.09110752708448276
平均 0.0889907161536207
东西南北 0.08544563452939656
贸易战 0.08411674935844828
北斗星 0.08398098766051723
益处 0.0826137029175862
珠穆朗玛峰 0.08039626442646552
一棵树 0.08022032587793103
串联 0.07878020445844829
驾驶证 0.07814134262948276
联通 0.07774030801198276
奇观 0.07530941101767243
凉爽 0.07398756601836207
诡异 0.07373767345913793
嘉宾 0.07361539212301724
衰老 0.07276803567094826
美国 0.07166545227
二哥 0.0715075481524138
南山 0.07

In [252]:
type(keywords)

list

## 9.2基于textrank算法

In [262]:
import jieba.analyse.textrank
keywords1=jieba.analyse.textrank(text123,allowPOS=('ns','nt','nz','a'))
for item in keywords1:
    print (item[0],item[1])

虚 拟
星 星
中 国
恐 龙
落 后
硅 酸
美 人
美 好
东 北
固 体
海 洋
城 市
美 国
黑 洞
泰 坦
真 实
珠 穆
北 斗
聪 明
霸 王
